In [10]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
pipeline = my_lib.Pipeline()
raw_df = pipeline.load_data()
raw_df = pipeline.encode_to_category(raw_df)
raw_df = pipeline.tokenize(raw_df)
raw_df.head()

,tweet,label,enterprise,is_irrelevant,token_list,clean_tweet,number_vector,input_vector
0,20 min line @apple store @short pump.,1,0,0,20 min line @apple store @short pump.,20 min line @apple store @short pump .,"[93, 437, 474, 207, 602, 159, 474, 396, 602, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Nueva tecnología convierte cualquier superfici...,3,1,1,nueva tecnología convierte cualquier superfici...,nueva tecnología convierte cualquier superfici...,"[159, 356, 523, 641, 603, 474, 177, 523, 214, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Some people should not post replies in #Google...,1,2,0,some people should not post replies in #google...,some people should not post replies in #google...,"[586, 129, 207, 523, 474, 168, 523, 129, 168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,I know a few others having same issue RT @Joel...,2,0,0,i know a few others having same issue rt @joel...,i know a few others having same issue rt @joel...,"[602, 474, 365, 159, 129, 432, 474, 603, 474, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"#Microsoft - We put the """"backwards"""" into bac...",2,1,0,"#microsoft - we put the """"backwards"""" into bac...","#microsoft - we put the "" "" backwards "" "" ...","[642, 207, 602, 214, 228, 129, 586, 129, 237, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [21]:
df = raw_df.loc[raw_df.label.isin([0,1,3,2])]
df = my_lib.processing.remove_useless_col(df, keep_col_list=["label", "clean_tweet"])
df_train_data, df_test_data = my_lib.processing.split_data(df)
print(f"SHAPE -> train : {df_train_data.shape}, test : {df_test_data.shape}")
df.head()


SHAPE -> train : (3338, 2), test : (835, 2)


,label,clean_tweet
0,1,20 min line @apple store @short pump .
1,3,nueva tecnología convierte cualquier superfici...
2,1,some people should not post replies in #google...
3,2,i know a few others having same issue rt @joel...
4,2,"#microsoft - we put the "" "" backwards "" "" ..."


In [53]:
a = []
a.extend(df["clean_tweet"].values)
b = [el.split(" ") for el in a]
c = set([my_el for my_list in b for my_el in my_list])

fp = open("/data/vocab.txt", "w")
print(*c, file=fp, sep="\n")


In [55]:
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file="/data/vocab.txt",
     do_lower_case=True)
print(len(tokenizer.vocab), tokenizer.vocab)

12314 OrderedDict([('', 0), ('redirect', 1), ('won', 2), ('#osea', 3), ('#inspired', 4), ('@hardware', 5), ('-)', 6), ('porque', 7), ('prise', 8), ('sad', 9), ('#funzioni', 10), ('تطابق', 11), ('aan', 12), ('upload', 13), ('blackeberry/iphone', 14), ('seguir', 15), ('creatividad', 16), ('stop', 17), ('#autopilot', 18), ('@hannahmccall1', 19), ('multimidia', 20), ('whoops', 21), ('05', 22), ('integrates', 23), ('//t', 24), ('truck', 25), ('usan', 26), ('@olsensn', 27), ('@mr_malie504', 28), ('seize', 29), ('funziona', 30), ('#cj20k', 31), ('$15', 32), ('achievement', 33), ('@desire2learn', 34), ('facebooku', 35), ('@kha1989led', 36), ('speaks', 37), ('officially', 38), ('búsquedas', 39), ('son', 40), ('#siri', 41), ('colin', 42), ('never', 43), ('por', 44), ('especialmente', 45), ('citou', 46), ('allowing', 47), ('w/operator', 48), ('ate', 49), ('gefällt', 50), ('unmindful', 51), ('foolish', 52), (')', 53), ('£1640', 54), ('name', 55), ('panfletos', 56), ('@dretoorn', 57), ('fundando', 

In [67]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

sentence1 = tf.ragged.constant([encode_sentence(s) for s in df["clean_tweet"].to_numpy()])



KeyError: '[UNK]'

array(['20 min line @apple store @short pump . ',
       'nueva tecnología convierte cualquier superficie en una pantalla multitactil .   <TOKEN_URL>  #microsoft #omnitouch',
       'some people should not post replies in #google+ threads .  their posts only continue to weaken their creditbility . ',
       ...,
       'i tried to explain why you would do  "  " the #twitter  "  "   :  &#8220 ; the twitter&#8221 ;  :   <TOKEN_URL>  @jefferypjacobs',
       "installed io5 - fine on ipad but wiped wife ' s iphone and restore from backup failed !  #bigtrouble #help @apple",
       '#microsoft #careers site is giving errors for any thing i search !  !  #weird  <TOKEN_URL>   <TOKEN_URL> '],
      dtype=object)